In [1]:
import cv2
import numpy as np

In [2]:
# load YOLO (use cv2.dnn)
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

In [3]:
with open('coco.names', 'r') as f_in:
    classes = [line.strip() for line in f_in.readlines()]
print(classes)
colors = np.random.uniform(0, 255, size=(len(classes), 3))

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [4]:
layer_names = net.getLayerNames()
outputlayers = [layer_names[i -1] for i in net.getUnconnectedOutLayers()]


In [17]:
img = cv2.imread('traffic2.jpg')



#img = cv2.resize(img, None, fx=2.40, fy=2.40)
height, width, channel = img.shape

blob = cv2.dnn.blobFromImage(img, 0.00382, (416, 416), (0,0,0), True, crop=False)

net.setInput(blob)
outs = net.forward(outputlayers)



#show info in screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:] # first for are coord
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.500:
            # objcet found
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            
            # cv2.circle(img, (center_x, center_y), 10, (0,255, 0))
            # draw rectangle
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            #cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

number_objects_detected = len(boxes)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.50, 0.400)#last 2 are thresholds

print(indexes)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        #color = colors[i]
        color = (0, 0, 255)
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, label, (x, y+15), font, 2, color, 2)
    
    
    
    
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('yolo_v3.jpg', img)

[ 18   7   9  79  94  30 100  83  41  62  23  54  57  14  51  67  26  49
  46  44  88  33  55  52  35  47  85  75  20  59]


True